Code that can be used to estimate descending button auctions
(see working paper on Windstream and the RDOF)
Ignacio Nunez, PhD Candidate in Economics, UT-Austin, ijnunez@utexas.edu

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels import api
from scipy import stats
from scipy import optimize
from scipy.optimize import minimize 
from scipy.optimize import show_options

# 1. Create sample with known parameters

In [ ]:
#competitors parameters
mu_1=0; 
mu_1x=0.5;
mu_1y=2;
st_1=0.5;

#Windstream parameters
mu_2=0;
mu_2x=0.5;
mu_2d=2;
st_2=0.5;
kappa=0.5;

In [ ]:
sample_size=500
# generate an independent variable 
x = np.linspace(-1, 2, sample_size)
# generate two normally distributed independent variables  residual
y = np.random.normal(0, 1, sample_size)
d = np.random.normal(0, 1, sample_size)

#Create sample
xi=np.random.normal(0, st_1, sample_size);
epsilon=xi*kappa+np.random.normal(0, st_2, sample_size);
X1 = mu_1+x*mu_1x+y*mu_1y+xi; #Competitors
X2 = mu_2+x*mu_2x+d*mu_2d+epsilon; #Windstream

df = pd.DataFrame({'x':x, 'y':y, 'd':d, 'Rivals':X1, 'Windstream':X2})
df.head()

In [ ]:
df['Observed_LastBid']=0
df['Censored']=0
df.loc[df['Rivals']>=df['Windstream'],'Observed_LastBid']=df.loc[df['Rivals']>=df['Windstream'],'Rivals']
df.loc[df['Rivals']>=df['Windstream'],'Censored']=1
df.loc[df['Rivals']<df['Windstream'],'Observed_LastBid']=df.loc[df['Rivals']<df['Windstream'],'Windstream']
df.head()

Sample=df[['x','y','d','Observed_LastBid','Censored']]
Sample['constant']=1
Sample.head()

# 2. Generate simulation draws for SMLE estimation

In [ ]:
sim_draws = np.random.normal(0, 1, 1000)
df_draws = pd.DataFrame({'draws':sim_draws})
df_draws['constant']=1
df_draws.head()

# 3. Define Likelihood function

In [ ]:
# Likelihood function
def Likelihood(parameters):
    mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa= parameters
    st_1=np.abs(st_1+0.001)
    st_2=np.abs(st_2+0.001)

    aux=Sample.copy()
    aux['Windstream']=mu_2*aux['constant']+mu_2x*aux['x']+mu_2d*aux['d'] #mean utility Windstream   
    aux['Rivals']=mu_1*aux['constant']+mu_1x*aux['x']+mu_1y*aux['y'] #mean utility rivals

    aux_unc = aux.loc[aux['Censored']==0,:] 
    aux_cen = aux.loc[aux['Censored']==1,:] 

    LL1 = np.sum(stats.norm.logpdf(((aux_cen['Observed_LastBid']-aux_cen['Rivals'])/st_1), 0, 1))
    LL2 = np.sum(stats.norm.logcdf(((aux_cen['Observed_LastBid']-aux_cen['Windstream']-kappa*(aux_cen['Observed_LastBid']-aux_cen['Rivals']))/st_2).to_numpy(), 0, 1))
    LL=LL1+LL2+aux_cen.shape[0]*np.log(1/st_1)
    #aux_unc['cdf_aux'] = np.nan
    aux_unc['cdf_aux'] = stats.norm.cdf((aux_unc['Observed_LastBid']-aux_unc['Rivals']), 0, st_1).tolist()
    LL=LL+(aux_unc.loc[aux_unc['cdf_aux']<=0.001,:].shape[0])*(np.log(0.00000001)+np.log(1/(st_2)))
    
    for index, row in aux_unc.loc[aux_unc['cdf_aux']>0.001,:].iterrows():
            aux_draws=df_draws.copy()
            aux_draws['draws']=aux_draws['draws']*st_1
            aux_draws=aux_draws.loc[(aux_draws['draws']<(row['Observed_LastBid']-row['Rivals'])),:]
            aux_draws_num=aux_draws.shape[0]
            if aux_draws_num>=1:
                LL3=np.sum(stats.norm.pdf(((row['Observed_LastBid']*aux_draws['constant']-row['Windstream']*aux_draws['constant']-kappa*aux_draws['draws'])/(st_2))))
                LL=LL+np.log((LL3/aux_draws_num)*stats.norm.cdf((row['Observed_LastBid']-row['Rivals']), 0, st_1)+0.000000001)+np.log(1/(st_2))
            else:
                LL=LL+np.log(0.00000001)+np.log(1/(st_2))

    neg_LL=-LL            
    return neg_LL 

# 4. Find parameters that maximize the log-likelihood

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
#model = minimize(Likelihood, np.array([1,1,1,1,1,1,1,1,0]), method='Nelder-Mead',options={'disp': True})
model = optimize.fmin(Likelihood, np.array([1,1,1,1,1,1,1,1,0]))

In [ ]:
model2 = optimize.fmin(Likelihood, model)
model2

In [ ]:
np.array([mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa])

In [ ]:
Likelihood(np.array([1,1,1,1,1,1,1,1,0]))
Likelihood(model2.x)
Likelihood(np.array([mu_1, mu_1x, mu_1y,st_1,mu_2,mu_2x,mu_2d,st_2,kappa]))